In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
# from natsort import natsorted
from glob import glob
from math import ceil
import hist
from hist import Hist
import mplhep as hep
plt.style.use(hep.style.CMS)
import boost_histogram as bh
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)

# Define DAQ Settings

In [ ]:
trigger_board = '0001'
# trigger_board_name = "ET2_EPIR_Pair1"
# readout_board_names = "b0_EPIR_Pair1_b1_BAR_4_b2_BAR_5_b3_EPIR_Pair4"
trigger_board_name = "ET2p01_BAR_4"
readout_board_names = "b0_BAR_4_b1_BAR_12_b2_BAR_13_b3_BAR_5"
readout_board_names = "b0_BAR_4_b1_BAR_12_b2_BAR_13_b3_CNM13HPK"
self_trigger_delay = 484
beam_downtime = False # default is False because we expect (hope?) for 100% uptime from accelerator
beam_off_time = "10-00" # time that beam is expected to turn off
beam_on_time = "11-30" # time that beam is expected to turn on
info = "offset_20" # additional info for the output name
LED_pages = ["0x0000","0x0004","0x0008","0x000C"] # LED pages to cycle through
active_channels = "0x000f"
# LED_pages = ["0x0000","0x0004","0x0008"] # LED pages to cycle through
# active_channels = "0x0007"
polarity = "0x0023" # "0x0023" or "0x0027"
hostname = "192.168.2.3"
ssd_path = "/media/daq/X9/DESY_Apr2024/ETROC-Data/"

testing = True

if not testing:
    total_time = 8 * 60 * 60
    daq_time = 300
    run_number = "2"
    run_options = "--compressed_binary --skip_translation"
else:
    total_time = 60
    daq_time = 15
    run_number = "rfsel_1_test_10"
    run_options = "--compressed_binary"
    #run_options = "--compressed_binary --skip_translation --suppress_fillers"

# Run DAQ

In [ ]:
trigger_bit_delay = int(trigger_board+'11'+format(self_trigger_delay, '010b'), base=2)
process_time = daq_time + 5
iterations = ceil(total_time/process_time)
LED_id = 0

for it in range(iterations):
    if beam_downtime:
        current_time = datetime.datetime.now().strftime("%H-%M")
        while ((current_time > beam_off_time) and (beam_on_time >= current_time)):
            print("Waiting for beam...")
            time.sleep(300)
            current_time = datetime.datetime.now().strftime("%H-%M")
    LED_page = LED_pages[LED_id]
    LED_id += 1
    if LED_id >= len(LED_pages): LED_id = 0
    reset_string = ""
    if(it==0): reset_string = "--check_valid_data_start"
    parser = parser_arguments.create_parser()
    (options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t {process_time} --run_name Run_{run_number} -o loop_{it} -v -w -s {LED_page} -p {polarity} -d {trigger_bit_delay} -a {active_channels} {run_options} --start_DAQ_pulse --stop_DAQ_pulse --ssd_path {ssd_path} {reset_string}".split())
    IPC_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Run_{run_number}_loop_{it}'))
    process.start()

    IPC_queue.put('memoFC Start Triggerbit BCR')
    while not IPC_queue.empty():
        pass
    time.sleep(daq_time)
    IPC_queue.put('stop DAQ')
    IPC_queue.put('memoFC Triggerbit')
    while not IPC_queue.empty():
        pass
    IPC_queue.put('allow threads to exit')
    process.join()

    del IPC_queue, process, parser